In [1]:
from pyspark.ml.linalg import Vectors,Vector

In [2]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('mllib').getOrCreate()

In [5]:
training = spark.createDataFrame([(0, "I heard about Spark and I love Spark",1),(1, "spark is fast",1),(2, "hadoop is good",0),(3,"hive is very well",0)],["id","text","label"])

定义Pipeline中的各个流水线阶段Pipeline Stage，包括转换器和评估其，具体的，包含tokenizer，hashingTF和lr

In [6]:
tokenizer=Tokenizer(inputCol="text",outputCol="words")
hashingTF=HashingTF(inputCol=tokenizer.getOutputCol(),outputCol="features")
lr=LogisticRegression(maxIter=10,regParam=0.001)

In [7]:
pipeline=Pipeline(stages=[tokenizer,hashingTF,lr])

现在构建的Pipeline本质上是一个Estimator，在它的fit方法运行之后，他将产生一个ePipelineModel,他是一个Transformer

In [8]:
model=pipeline.fit(training)

构建测试数据

In [13]:
test=spark.createDataFrame([(4,"spark is the best"),(5,"Hbase is good"),(6,"this is spark"),(7,"it is flink")],["id","text"])

调用训练好的PipelineModel,的transform方法，让测试数据按照顺序通过拟合的流水线，产生预测结果

In [14]:
prediction = model.transform(test)

In [15]:
selected=prediction.select("id","text","probability","prediction")

In [16]:
selected.show(truncate=False)

+---+-----------------+----------------------------------------+----------+
|id |text             |probability                             |prediction|
+---+-----------------+----------------------------------------+----------+
|4  |spark is the best|[0.22857839831078508,0.7714216016892149]|1.0       |
|5  |Hbase is good    |[0.9581222139410415,0.04187778605895862]|0.0       |
|6  |this is spark    |[0.22857839831078508,0.7714216016892149]|1.0       |
|7  |it is flink      |[0.5910580523621201,0.40894194763787994]|0.0       |
+---+-----------------+----------------------------------------+----------+

